In [2]:
import pandas as pd
import numpy as np
from math import exp
from keras.utils import to_categorical
import logging

input data:
shape = (2,6)

Layer 1

$L1 = h1(input·W1+b1)$

Layer 2

$L2 = h2(L1·W2+b2)$

Output layer

$output = h3(L2·W3+b3)$

activation function
sigmoid function:
h(x)=1/(1+exp(-x))

In [3]:
# sigmoid function
def sigmoid(x):
    return 1/float(1+exp(-x))

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [4]:
# activation
def h(input_array):
    shape = input_array.shape
    return np.array([sigmoid(item) for row in input_array for item in row]).reshape(shape[0],shape[1])

In [5]:
# L1：3个神经元
# L2：6个神经元
# L3：2个神经元
input_data = np.array([[1,2,3,4,5,6],[1,3,5,7,9,11]]) # shape = (2,6) (样本数目，输入数据维数)
W1 = np.array([
        [2,2,2],
        [2,2,2],
        [2,2,2],
        [2,2,2],
        [2,2,2],
        [2,2,2]
    ]) # shape = (输入数据维数,L1神经元数目),
b1 = np.array([[3,3,3]]) # shape = （1，L1神经元数目）
W2 = np.array([
            [1,2,3,3,2,1], 
            [1,2,3,3,2,1],
            [1,2,3,3,2,1]
    ]) # shape = (上一层神经元数目，L2神经元数目) 
b2 = np.array([[7,6,5,4,3,2]]) # shape = （1，L2神经元数目）
W3 =  np.array([ 
            [1,2],
            [3,3],
            [2,3],
            [2,1],
            [1,1],
            [4,5]
    ])# shape = (上一层神经元数目，输出层神经元个数)
b3 = np.array([8,9])# shape = (1,输出层神经元个数)

In [6]:
L1 = h( np.dot(input_data,W1) + b1 )
L1.shape # shape = (样本数目，L1神经元数目)

(2, 3)

In [7]:
L2 = h( np.dot(L1,W2) + b2 )
L2.shape # shape = (样本数目，L2神经元数目)

(2, 6)

In [8]:
L3 = h( np.dot(L2,W3) + b3 )
L3.shape # shape = (样本数目，输出层神经元数目)3u

(2, 2)

In [9]:
L3

array([[1., 1.],
       [1., 1.]])

In [25]:
def forward(input_data,params):
    W1 = params['W1']
    b1 = params['b1']
    W2 = params['W2']
    b2 = params['b2']
    W3 = params['W3']
    b3 = params['b3']
    L1 = h( np.dot(input_data,W1) + b1 )
    L2 = h( np.dot(L1,W2) + b2 ) 
    out = h( np.dot(L2,W3) + b3 )
    return out

In [16]:
input_data

array([[ 1,  2,  3,  4,  5,  6],
       [ 1,  3,  5,  7,  9, 11]])

In [17]:
inputs = input_data[0,].astype('float64')
inputs

array([1., 2., 3., 4., 5., 6.])

In [18]:
inputs0 = inputs.copy()
inputs0

array([1., 2., 3., 4., 5., 6.])

In [19]:
inputs[0]=inputs[0]+0.01
inputs

array([1.01, 2.  , 3.  , 4.  , 5.  , 6.  ])

In [20]:
inputs0

array([1., 2., 3., 4., 5., 6.])

In [21]:
answer = np.array([0.0,1.0])

In [22]:
def mse(pred_out,answer):
    return 0.5*np.sum((pred_out-answer)**2)
    
def cross_entropy(pred_out,answer):
    return -np.sum(answer * np.log(pred_out),axis=1)# 按行求和

def loss_function(pred_out):
    answer = np.array([0.0,1.0])
    N = len(answer)
    return np.sum(cross_entropy(pred_out,answer))/float(N)


In [23]:
loss_function(np.array([[0.1,0.9],[0.2,0.8],[0.5,0.5]]))

0.5108256237659906

In [26]:
def gradient(inputs,params):
    N = params.size
    gradient = np.array([0]*N)
    delda_x = 0.01
    for i in range(N):
        new_params = params.copy()
        new_params[i] = new_params[i]+delda_x
        delda_y = loss_function(forward(inputs,params))-loss_function(forward(inputs,new_params)) ###
        gradient[i] = delda_y/float(delda_x)
    return gradient

def f(W):
    return loss_function(forward(inputs,params))
    
dW = gradient(f,np.array([[1,2,3],[4,5,6]]))

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [23]:
gradient(inputs) 

array([0, 0, 0, 0, 0, 0])

In [25]:
def gradient_descent(params,lr=0.01):
    grads = gradient(params)
    params = params - lr*grads
    return params

In [27]:
a=np.array([[1,2,3],[4,5,6]])  
a[4]

SyntaxError: invalid syntax (<ipython-input-27-2d7330a56723>, line 2)

In [28]:
## 1007 practice 
import sys
sys.path.append('/mnt/d/home/git/Deep_Learning_from_scratch')

from common.functions import *

def numerical_gradient(f, x):# 2d
    h = 1e-4 # 0.0001t5
    grad = np.zeros_like(x)

    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)

        x[idx] = tmp_val - h
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)

        x[idx] = tmp_val # 还原值
        it.iternext()

    return grad

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size,
                weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size) 
        self.params['W2'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        #print(x.shape,W1.shape)
        #print (x)
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y

    def loss(self, x, t):
        'x: 输入数据， t: 监督数据'
        y = self.predict(x)
        return cross_entropy_error(y,t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        #print(x)
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads


In [29]:
net = TwoLayerNet(6,3,2)

In [30]:
net.params['W1'].shape

(6, 3)

In [31]:
grads = net.numerical_gradient(input_data[0],answer)
grads

{'W1': array([[ 0.00253716, -0.00114845, -0.00271986],
        [ 0.00507431, -0.0022969 , -0.00543972],
        [ 0.00761147, -0.00344535, -0.00815958],
        [ 0.01014863, -0.0045938 , -0.01087943],
        [ 0.01268578, -0.00574224, -0.01359929],
        [ 0.01522294, -0.00689069, -0.01631915]]),
 'b1': array([ 0.00253716, -0.00114845, -0.00271986]),
 'W2': array([[ 0.24667025, -0.24667025],
        [ 0.2566529 , -0.2566529 ],
        [ 0.25272042, -0.25272042]]),
 'b2': array([ 0.4985653, -0.4985653])}

In [33]:
lr = 0.01
new_params = {}
for k,v in grads.items():
    new_params[k] =  net.params[k]-lr * v
new_params

{'W1': array([[ 0.00896974, -0.0204518 , -0.01576446],
        [ 0.00317951,  0.00271875, -0.02091667],
        [ 0.00285241,  0.01633186, -0.01595991],
        [-0.00429138,  0.00030224,  0.01851162],
        [-0.00730417,  0.00944017,  0.00359874],
        [ 0.00108857, -0.00369951,  0.00724925]]),
 'b1': array([-2.53715659e-05,  1.14844901e-05,  2.71985871e-05]),
 'W2': array([[ 0.00887157, -0.00655292],
        [-0.01401695,  0.0003382 ],
        [-0.0223129 ,  0.00456714]]),
 'b2': array([-0.00498565,  0.00498565])}

In [34]:
net.params = new_params 

20201011 practice

class MyTwoLayerNet
attributes:
    params

In [ ]:
weight_init_std = 0.01
init_params = {}
init_params['W1'] = weight_init_std * \
                    np.random.randn(input_size, hidden_size)
init_params['b1'] = np.zeros(hidden_size) 
init_params['W2'] = weight_init_std * \
                    np.random.randn(hidden_size, output_size)
init_params['b2'] = np.zeros(output_size)
        

In [36]:
np.random.randn(1,2)

array([[-0.06838368, -1.14292484]])

In [131]:
import sys
sys.path.append('/mnt/d/home/git/Deep_Learning_from_scratch')

from common.functions import *
class MyTwoLayerNet():
    
    def __init__(self,input_size,hidden_size,output_size,weight_init_std=0.01):
        self.params = {}
        # hidden layer
        self.params['W1'] = weight_init_std*np.random.randn(input_size,hidden_size) # shape = (input size, hidden size)
        self.params['b1'] = weight_init_std*np.random.randn(1,hidden_size) # shape = (1,hidden szie)
        # output layer
        self.params['W2'] = weight_init_std*np.random.randn(hidden_size,output_size) # shape = (hidden size, output size)
        self.params['b2'] = weight_init_std*np.random.randn(1,output_size)# shape = (1, output size)
        
    def predict(self,input_data):
        hidden_layer = sigmoid( np.dot(input_data,self.params['W1'])+self.params['b1'] )
        output = softmax(  np.dot(hidden_layer,self.params['W2'])+self.params['b2'] )
        return output 
    
    def _numerical_gradient(self,f,x ):
        h = 1e-4 # 0.0001t5
        grad = np.zeros_like(x)

        it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            idx = it.multi_index
            tmp_val = x[idx]
            x[idx] = float(tmp_val) + h
            fxh1 = f(x) # f(x+h)

            x[idx] = tmp_val - h
            fxh2 = f(x) # f(x-h) 
            grad[idx] = (fxh1 - fxh2) / (2*h)

            x[idx] = tmp_val # 还原值
            it.iternext()

        return grad
    
    def gradient_descent(self, input_data,label,lr=0.001):
        def loss(x, t):
            'x: 输入数据， t: 监督数据'
            y = self.predict(x)
            return cross_entropy_error(y,t)
        def loss_W(W):
            return loss(input_data,label)
        
        self.params['W1'] -= lr*self._numerical_gradient(loss_W,self.params['W1'])
        self.params['b1'] -= lr*self._numerical_gradient(loss_W,self.params['b1'])
        self.params['W2'] -= lr*self._numerical_gradient(loss_W,self.params['W2'])
        self.params['b2'] -= lr*self._numerical_gradient(loss_W,self.params['b2'])
        
    def train(self,input_data,label,epoch=5):
        for i in range(epoch):
            self.gradient_descent(input_data,label)
            logging.info('epoch {} done.'.format(i))
    
    def accuracy(self, y, t):
#        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y==t) / float(y.shape[0])
        return accuracy
    
        

In [82]:
net = MyTwoLayerNet(6,10,2)
net.train(input_data,np.array([1,0]))
pred_val = net.predict(input_data)
pred_val
#net.eval(pred_val,test_label)

5


array([[0.50874344, 0.49125656],
       [0.5087993 , 0.4912007 ]])

__test model on MNIST data set__

In [86]:
data = pd.read_csv('/mnt/d/home/git/kaggle-digit-recognizer/input/train.csv',sep=',')
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
data.shape

(42000, 785)

In [133]:
train = data.iloc[:1000]
test = data.iloc[1000:1100]
train_data = train.iloc[:,1:]
train_label = to_categorical(train.iloc[:,0])
test_data = test.iloc[:,1:]
test_label = to_categorical(test.iloc[:,0])

In [134]:
net = MyTwoLayerNet(784,10,10)
net.train(train_data,train_label)
pred_val = net.predict(test_data)
net.accuracy(pred_val,test_label)
#net.eval(pred_val,test_label)

0.07